In [1]:
import kaolin
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import polyscope as ps

# import diffvoronoi
import pygdel3d
import sdfpred_utils.sdfpred_utils as su
import sdfpred_utils.loss_functions as lf
from pytorch3d.loss import chamfer_distance
from pytorch3d.ops import knn_points, knn_gather
import torch
from torch import nn

# cuda devices
device = torch.device("cuda:0")
print("Using device: ", torch.cuda.get_device_name(device))

# Improve reproducibility
torch.manual_seed(69)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(69)

input_dims = 3
lr_sites = 0.005
# lr_model = 0.00001
destination = "./images/autograd/End2End_DCCVT_interpolSDF_trueCVT/"
model_trained_it = ""
# ROOT_DIR = "/home/wylliam/dev/Kyushu_experiments"
ROOT_DIR = "/home/beltegeuse/projects/Voronoi/Kyushu_experiments"


# mesh = ["sphere"]

# mesh = ["gargoyle", "/home/wylliam/dev/Kyushu_experiments/data/gargoyle"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-24-18-16-03/gargoyle/gargoyle/trained_models/model{model_trained_it}.pth"

mesh = [
    "gargoyle",
    f"{ROOT_DIR}/mesh/thingi32/64764",
]
trained_model_path = f"{ROOT_DIR}/hotspots_model/thingi32/64764.pth"

# mesh = ["zombie", f"{ROOT_DIR}/mesh/thingi32/398259"]
# trained_model_path = f"{ROOT_DIR}/hotspots_model/thingi32/398259.pth"

# mesh = ["gargoyle_unconverged", "/home/wylliam/dev/Kyushu_experiments/mesh/gargoyle_unconverged"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-24-18-16-03/gargoyle/gargoyle/trained_models/model_500.pth"


# mesh = ["chair", "/home/wylliam/dev/Kyushu_experiments/data/chair"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-05-02-17-56-25/chair/chair/trained_models/model{model_trained_it}.pth"
# #
# mesh = ["bunny", "/home/wylliam/dev/Kyushu_experiments/data/bunny"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-25-17-32-49/bunny/bunny/trained_models/model{model_trained_it}.pth"


Using device:  NVIDIA GeForce RTX 3080


In [2]:
num_centroids = 32**3
grid = 32  # 128
print("Creating new sites")
noise_scale = 0.005
domain_limit = 1
x = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids ** (1 / 3))))
y = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids ** (1 / 3))))
z = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids ** (1 / 3))))
meshgrid = torch.meshgrid(x, y, z)
meshgrid = torch.stack(meshgrid, dim=3).view(-1, 3)

# add noise to meshgrid
meshgrid += torch.randn_like(meshgrid) * noise_scale


sites = meshgrid.to(device, dtype=torch.float32).requires_grad_(True)

print("Sites shape: ", sites.shape)
print("Sites: ", sites[0])
ps.init()


Creating new sites
Sites shape:  torch.Size([32768, 3])
Sites:  tensor([-1.0027, -1.0065, -0.9978], device='cuda:0', grad_fn=<SelectBackward0>)
[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 575.64.03


/home/beltegeuse/projects/Voronoi/Kyushu_experiments/.venv/lib/python3.13/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
# LOAD MODEL WITH HOTSPOT

import sys

if mesh[0] != "sphere":
    sys.path.append("3rdparty/HotSpot")
    from dataset import shape_3d
    import models.Net as Net

    loss_type = "igr_w_heat"
    loss_weights = [350, 0, 0, 1, 0, 0, 20]

    train_set = shape_3d.ReconDataset(
        file_path=mesh[1] + ".ply",
        n_points=grid * grid * 150,  # 15000, #args.n_points,
        n_samples=10001,  # args.n_iterations,
        grid_res=256,  # args.grid_res,
        grid_range=1.1,  # args.grid_range,
        sample_type="uniform_central_gaussian",  # args.nonmnfld_sample_type,
        sampling_std=0.5,  # args.nonmnfld_sample_std,
        n_random_samples=7500,  # args.n_random_samples,
        resample=True,
        compute_sal_dist_gt=(True if "sal" in loss_type and loss_weights[5] > 0 else False),
        scale_method="mean",  # "mean" #args.pcd_scale_method,
    )

    model = Net.Network(
        latent_size=0,  # args.latent_size,
        in_dim=3,
        decoder_hidden_dim=128,  # args.decoder_hidden_dim,
        nl="sine",  # args.nl,
        encoder_type="none",  # args.encoder_type,
        decoder_n_hidden_layers=5,  # args.decoder_n_hidden_layers,
        neuron_type="quadratic",  # args.neuron_type,
        init_type="mfgi",  # args.init_type,
        sphere_init_params=[1.6, 0.1],  # args.sphere_init_params,
        n_repeat_period=30,  # args.n_repeat_period,
    )
    model.to(device)

    ######
    test_dataloader = torch.utils.data.DataLoader(
        train_set, batch_size=1, shuffle=False, num_workers=0, pin_memory=False
    )
    test_data = next(iter(test_dataloader))
    mnfld_points = test_data["mnfld_points"].to(device)

    # add noise to mnfld_points
    # mnfld_points += torch.randn_like(mnfld_points) * noise_scale * 2

    mnfld_points.requires_grad_()
    print("mnfld_points shape: ", mnfld_points.shape)
    if torch.cuda.is_available():
        map_location = torch.device("cuda")
    else:
        map_location = torch.device("cpu")
    model.load_state_dict(torch.load(trained_model_path, weights_only=True, map_location=map_location))
    sdf0 = model(sites)

else:

    def sphere_sdf(points: torch.Tensor, center: torch.Tensor, radius: float) -> torch.Tensor:
        """
        Compute the SDF of a sphere at given 3D points.

        Args:
            points: (N, 3) tensor of 3D query points
            center: (3,) tensor specifying the center of the sphere
            radius: float, radius of the sphere

        Returns:
            sdf: (N,) tensor of signed distances
        """
        return torch.norm(points - center, dim=-1) - radius

    def sphere_sdf_with_noise(
        points: torch.Tensor, center: torch.Tensor, radius: float, noise_amplitude=0.05
    ) -> torch.Tensor:
        """
        Sphere SDF with smooth directional noise added near the surface.

        Args:
            points: (N, 3)
            center: (3,)
            radius: float
            noise_amplitude: float

        Returns:
            sdf: (N,)
        """
        rel = points - center
        norm = torch.norm(rel, dim=-1)  # (N,)
        base_sdf = norm - radius  # (N,)

        # Smooth periodic noise based on direction
        unit_dir = rel / (norm.unsqueeze(-1) + 1e-9)  # (N,3)
        noise = torch.sin(10 * unit_dir[:, 0]) * torch.sin(10 * unit_dir[:, 1]) * torch.sin(10 * unit_dir[:, 2])

        # Weight noise so it mostly affects surface area
        falloff = torch.exp(-20 * (base_sdf**2))  # (N,) ~1 near surface, ~0 far
        sdf = base_sdf + noise_amplitude * noise * falloff

        return sdf

    # generate points on the sphere
    mnfld_points = torch.randn(grid * grid * 150, 3, device=device)
    mnfld_points = mnfld_points / torch.norm(mnfld_points, dim=-1, keepdim=True) * 0.5
    mnfld_points = mnfld_points.unsqueeze(0).requires_grad_()
    sdf0 = sphere_sdf(sites, torch.zeros(3).to(device), 0.50)
    # sdf0 = sphere_sdf_with_noise(sites, torch.zeros(3).to(device), 0.50, noise_amplitude=0.1)

# # add mnfld points with random noise to sites
# N = mnfld_points.squeeze(0).shape[0]
# num_samples = 18**3 - (num_centroids)
# idx = torch.randint(0, N, (num_samples,))
# sampled = mnfld_points.squeeze(0)[idx]
# perturbed = sampled + (torch.rand_like(sampled) - 0.5) * noise_scale * 10
# sites = torch.cat((sites, perturbed), dim=0)
# sdf0 = model(sites)

# make sites a leaf tensor
sites = sites.detach().requires_grad_()
print(sites.dtype)
print(sites.shape)
print(f"Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")


sdf0 = sdf0.detach().squeeze(-1).requires_grad_()
print(sdf0.shape)
print(sdf0.is_leaf)

# Show now in polyscope
ps.init()
ps.set_up_dir("z_up")
ps.register_point_cloud("point cloud", mnfld_points.squeeze().detach().cpu().numpy())
ps.show()


mnfld_points shape:  torch.Size([1, 9600, 3])
torch.float32
torch.Size([32768, 3])
Allocated: 430.440448 MB, Reserved: 444.596224 MB
torch.Size([32768])
True


In [9]:
import open3d as o3d

# Visualize the point sampled
pcd = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(mnfld_points.squeeze().detach().cpu().numpy()))
# o3d.visualization.draw_geometries([pcd])
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(100)

# radii = [0.005, 0.01, 0.02, 0.04]
# radii = [r * 10 for r in radii]  # scale down the radii for better visualization
# print("Radii for ball pivoting:", radii)
# rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd, o3d.utility.DoubleVector(radii))
# o3d.visualization.draw_geometries([pcd, rec_mesh],  point_show_normal=True)

def compute_poisson_from_pcd(pcd, depth=9, remove_density=0.0):
    """
    Compute Poisson surface reconstruction from a point cloud.
    
    Args:
        pcd: Open3D PointCloud object
        depth: Depth of the Poisson reconstruction
    
    Returns:
        mesh: Open3D TriangleMesh object
        densities: Densities of the reconstructed mesh
    """
    
    print('run Poisson surface reconstruction')
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=9)
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
        (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    # Gray colors for the mesh
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(np.ones((len(mesh.vertices), 3)) * 0.5)

    #o3d.utility.Vector3dVector(density_colors)
    if remove_density > 0.0:
        # Remove vertices with density below the threshold
        vertices_to_remove = densities < remove_density
        density_mesh.remove_vertices_by_mask(vertices_to_remove)
    print('done Poisson surface reconstruction')

    return density_mesh

density_mesh = compute_poisson_from_pcd(pcd, depth=9, remove_density=0.0)
# o3d.visualization.draw_geometries([density_mesh], mesh_show_back_face=True)


run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 9600 / 9519
[Open3D DEBUG] #   Got kernel density: 0.023105859756469727 (s), 2005.41015625 (MB) / 2005.41015625 (MB) / 2353 (MB)
[Open3D DEBUG] #     Got normal field: 0.026742935180664062 (s), 2006.97265625 (MB) / 2006.97265625 (MB) / 2353 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.044675e-04 / 2.922888e+00
[Open3D DEBUG] #       Finalized tree: 0.06024789810180664 (s), 2007.12890625 (MB) / 2007.12890625 (MB) / 2353 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.053704023361206055 (s), 2007.12890625 (MB) / 2007.12890625 (MB) / 2353 (MB)
[Open3D DEBUG] #Set point constraints: 0.00828099250793457 (s), 2007.12890625 (MB) / 2007.12890625 (MB) / 2353 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 376685 / 183104 / 247393
[Open3D DEBUG] Memory Usage: 2007.129 MB
[Open3D DEBUG] # Linear system solved: 0.17824983596801758 (s), 2007.12890625 (MB) / 2007.12890625 (MB) / 2353 (MB)Cycle[0] Depth[0/9]:

In [5]:
def show_mesh_in_polyscope(density_mesh, name="mesh"):
    """
    Show the mesh in Polyscope.
    
    Args:
        density_mesh: Open3D TriangleMesh object
    """    
    # Convert Open3D mesh to Polyscope format
    vertices = np.asarray(density_mesh.vertices)
    faces = np.asarray(density_mesh.triangles)
    
    ps.register_surface_mesh(name, vertices, faces)
    ps.show()

show_mesh_in_polyscope(density_mesh, name="simple_mesh")

In [6]:
import wn_treecode
import time
import tqdm
import torch.nn.functional as F

ITERS = 40

mnfld_points = mnfld_points.squeeze()

normals = torch.zeros_like(mnfld_points, device=mnfld_points.device).contiguous().float()
b = torch.ones(mnfld_points.shape[0], 1, device=mnfld_points.device) * 0.5
widths = torch.ones_like(mnfld_points[:, 0], device=mnfld_points.device)

wn_func = wn_treecode.WindingNumberTreecode(mnfld_points)

wsmin, wsmax = [0.002, 0.016]

time_iter_start = time.time()
if wn_func.is_cuda:
    torch.cuda.synchronize(device=None)
with torch.no_grad():
    bar = tqdm.tqdm(range(ITERS))

    for i in bar:
        width_scale = wsmin + ((ITERS-1-i) / ((ITERS-1))) * (wsmax - wsmin)
        # width_scale = args.wsmin + 0.5 * (args.wsmax - args.wsmin) * (1 + math.cos(i/(args.iters-1) * math.pi))
        
        # grad step
        A_mu = wn_func.forward_A(normals, widths * width_scale)
        AT_A_mu = wn_func.forward_AT(A_mu, widths * width_scale)
        r = wn_func.forward_AT(b, widths * width_scale) - AT_A_mu
        A_r = wn_func.forward_A(r, widths * width_scale)
        alpha = (r * r).sum() / (A_r * A_r).sum()
        normals = normals + alpha * r

        # WNNC step
        out_normals = wn_func.forward_G(normals, widths * width_scale)

        # rescale
        out_normals = F.normalize(out_normals, dim=-1).contiguous()
        normals_len = torch.linalg.norm(normals, dim=-1, keepdim=True)
        normals = out_normals.clone() * normals_len

if wn_func.is_cuda:
    torch.cuda.synchronize(device=None)
time_iter_end = time.time()
print(f'[LOG] time_main: {time_iter_end - time_iter_start}')

num_nodes: 16417, num_leaves: 9600, tree depth: 14
tree_depth*num_points: 134400, stdvec_node2point_index.size(): 83879


100%|██████████| 40/40 [00:00<00:00, 87.55it/s] 

[LOG] time_main: 0.511730432510376


In [ ]:
pcd.normals = o3d.utility.Vector3dVector(out_normals.detach().cpu().numpy())
# o3d.visualization.draw_geometries([pcd], point_show_normal=True)

mesh = compute_poisson_from_pcd(pcd, depth=10)
show_mesh_in_polyscope(mesh, name="wnnc_mesh")


run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 9600 / 9519
[Open3D DEBUG] #   Got kernel density: 0.022270917892456055 (s), 2016.40234375 (MB) / 2016.40234375 (MB) / 2353 (MB)
[Open3D DEBUG] #     Got normal field: 0.018496990203857422 (s), 2018.27734375 (MB) / 2018.27734375 (MB) / 2353 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.044676e-04 / 2.922889e+00
[Open3D DEBUG] #       Finalized tree: 0.05953097343444824 (s), 2018.27734375 (MB) / 2018.27734375 (MB) / 2353 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.2087080478668213 (s), 2018.27734375 (MB) / 2018.27734375 (MB) / 2353 (MB)
[Open3D DEBUG] #Set point constraints: 0.04207205772399902 (s), 2018.27734375 (MB) / 2018.27734375 (MB) / 2353 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 376685 / 183104 / 247393
[Open3D DEBUG] Memory Usage: 2018.277 MB
Cycle[0] Depth[0/9]:	Updated constraints / Got system / Solved in:  0.000 /  0.000 /  0.000	(2018.277 MB)	Nodes: 8
CG: 1.0528e+00 -> 1.05

KeyboardInterrupt: 